# Simulartcra

In [1]:
import logging
from datetime import datetime, timedelta
from typing import List
import collections
import inspect
import tenacity
from termcolor import colored
import os
import io
import warnings
from PIL import Image
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
from langchain.output_parsers import RegexParser
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)
from dotenv import load_dotenv

logging.basicConfig(level=logging.ERROR)

load_dotenv()
os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'

In [2]:
USER_NAME = "schoi"  # The name you want to use when interviewing the agent.
LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.
# Set up our connection to the API.
stability_api = client.StabilityInference(
    key=os.environ['STABILITY_KEY'], # API Key reference.
    verbose=True, # Print debug messages.
    engine="stable-diffusion-xl-1024-v1-0", # Set the engine to use for generation.
    # Check out the following link for a list of available engines: https://platform.stability.ai/docs/features/api-parameters#engine
)

INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443


In [3]:
import math
import faiss


def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [4]:
## stability ai art
def generate_art(prompt, turn, seed=4253978046, init_img=None, mask=None, samples=1, steps=50, cfg_scale=8.0, width=1024, height=1024, sampler=generation.SAMPLER_K_DPMPP_2M):
    answers = stability_api.generate(
        prompt=prompt,
        init_image=init_img, # Assign our previously generated img as our Initial Image for transformation.
        mask_image=mask,
        seed=seed, # If a seed is provided, the resulting generated image will be deterministic.
                        # What this means is that as long as all generation parameters remain the same, you can always recall the same image simply by generating it again.
                        # Note: This isn't quite the case for Clip Guided generations, which we'll tackle in a future example notebook.
        steps=steps, # Amount of inference steps performed on image generation. Defaults to 30. 
        cfg_scale=cfg_scale, # Influences how strongly your generation is guided to match your prompt.
        width=width, # Generation width, defaults to 512 if not included.
        height=height, # Generation height, defaults to 512 if not included.
        samples=samples, # Number of images to generate, defaults to 1 if not included.
        sampler=sampler
    )
    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn(
                    "Your request activated the API's safety filters and could not be processed."
                    "Please modify the prompt and try again.")
            if artifact.type == generation.ARTIFACT_IMAGE:
                img = Image.open(io.BytesIO(artifact.binary))
                img.save(f"result_{turn:02d}.png") # Save our generated images with their seed number as the filename.

## Creating the agents

In [5]:
alices_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

alice = GenerativeAgent(
    name="Alice",
    age=25,
    traits="likes classical art, action driven",  # You can add more persistent traits here
    status="wants to create art",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=alices_memory,
)

bobs_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

bob = GenerativeAgent(
    name="Bob",
    age=25,
    traits="likes modern art, action driven",  # You can add more persistent traits here
    status="wants to create art",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=bobs_memory,
)

# We can add memories directly to the memory object

observations = [
    " is going to collaborating on creating art",
    " can create new art by saying GENERATE",
    " can iterate on the existing art by saying ITERATE",
]
for observation in observations:
    alice.memory.add_memory("Alice" + observation)
    bob.memory.add_memory("Bob" + observation)

In [6]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [7]:
# interview_agent(alice, "What do you like to do?")

In [8]:
# interview_agent(bob, "What do you like to do?")

## Main Event Loop

In [9]:
def main(agents: List[GenerativeAgent], initial_observation: str, max_turns=10) -> None:
    """
    Simulate Art Creation
    TODO: Create a canvas environment for gymnasium or something similar
    I am no good at prompt engineering for actions. it worked once
    """
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    prev_generated = turns
    generated = False
    while True:
        if turns > max_turns:
            break
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            generate = observation.find("GENERATE")
            iterate = observation.find("ITERATE")
            if generate >= 0 and not generated:
                generate_art(observation, turns)
                prev_generated = turns
                generated = True
                for agent in agents:
                    agent.memory.add_memory("An initial art piece was generated, I should now only say ITERATE with an improvement")
            if iterate >= 0:
                generate_art(observation, turns, init_img=f"result_{prev_generated:02d}.png")
                prev_generated = turns
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [10]:
agents=[alice, bob]
initial_observation = """
Alice said: Hello Bob! Are you ready to create the art piece for the demo? The theme is 'imagination' lets generate the art as soon as possible and iterate.
"""
main(agents, initial_observation)

Bob said "Yes, I'm ready! Let's generate the art piece for the demo with the theme 'imagination' and then iterate on it."
Alice said "Great! Let's start by generating the art piece with the theme 'imagination'. I'm excited to see what we can create together!"
Bob said "I'm excited too! Let's get started and see what we can come up with. I'll go ahead and generate the art piece with the theme 'imagination'."
Alice said "That sounds great, Bob! I can't wait to see what you generate. Let's create something truly imaginative together!"
Bob said "I'm glad you're excited, Alice! I'm ready to generate the art piece with the theme 'imagination' now. Let's create something truly imaginative together!"
Alice said "That's great, Bob! I'm really looking forward to seeing what we can come up with. Let's create something truly imaginative together!"


KeyboardInterrupt: 